In [1]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json
import re
from geopy.geocoders import Nominatim

In [2]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [3]:
import ast

In [4]:
df = pd.read_csv('../data/data_tripadvisor.csv', index_col=0)
df.dropna(how = "all", axis = 0, inplace = True)

In [5]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3
indice,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España"


In [6]:
df.dtypes

name           object
address        object
type           object
ranking        object
price          object
direccion      object
comunidad      object
street         object
street_2       object
coordinates    object
street_3       object
dtype: object

In [7]:
def ast_literal (x):
    try:
        return ast.literal_eval(x)
    except:
        return x

In [8]:
df.coordinates = df.coordinates.apply(ast_literal)

In [9]:
def turn_around(x):
    b = (x[1], x[0])
    return list(b)

In [10]:
df["coordinates_v2"] = df.coordinates.apply(turn_around)

In [11]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2
indice,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]"


In [12]:
type(df.coordinates_v2[0])

list

In [13]:
def type_point(x):
    return {"type":"Point", "coordinates": [x[1], x[0]]}

In [14]:
df["type_point"] = df["coordinates"].apply(type_point)

In [15]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
indice,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4..."
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ..."


In [16]:
type(df.type_point[0])

dict

In [17]:
def get_latitude (x):
    return x[0]

In [18]:
def get_longitude (x):
    return x[1]

In [19]:
df["longitude"] = df["coordinates"].apply(get_longitude)

In [20]:
df["latitude"] = df["coordinates"].apply(get_latitude)

In [21]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,longitude,latitude
indice,,,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4...",-3.606505,40.028851
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ...",-3.590871,40.031684


In [22]:
def delete_hashtag(x):
    x = str(x).replace('#', "")
    return x

In [23]:
df["ranking"] = df.ranking.apply(delete_hashtag)

In [24]:
def delete_string(x):
    x = str(x).replace('restaurantes en Madrid', "")
    return x

In [25]:
df["ranking"] = df.ranking.apply(delete_string)

In [26]:
df


,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,longitude,latitude
indice,,,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,549 de 10.409,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4...",-3.606505,40.028851
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,34 de 10.414,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ...",-3.590871,40.031684
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,650 de 10.414,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]","{'type': 'Point', 'coordinates': [-3.6142713, ...",-3.614271,40.039589
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,311 de 10.414,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]","{'type': 'Point', 'coordinates': [-3.6142713, ...",-3.614271,40.039589
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]","{'type': 'Point', 'coordinates': [-3.4252641, ...",-3.425264,40.138485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199.0,Restaurante El Senador,"Plaza Marina Espanola 2, 28013 Madrid España",Mediterránea,772 de 10.409,€€ - €€€,Plaza Marina Espanola 2,Madrid España,Plaza Marina Espanola 2,"Plaza Marina Espanola 2, Madrid España","(40.4206521, -3.7114917)","Plaza de la Marina Espanola 2, Madrid España","[-3.7114917, 40.4206521]","{'type': 'Point', 'coordinates': [-3.7114917, ...",-3.711492,40.420652
415.0,Bar Cerveriz,Plaza San Miguel 2 Frente al Mercado de San Mi...,Mediterránea,674 de 10.409,€,Plaza San Miguel 2 Frente al Mercado de San Mi...,Madrid España,Plaza San Miguel 2,"Plaza San Miguel 2, Madrid España","(40.4156034, -3.7092267)","Plaza San Miguel 2, Madrid","[-3.7092267, 40.4156034]","{'type': 'Point', 'coordinates': [-3.7092267, ...",-3.709227,40.415603
292.0,Cerveceria Alemana,"Plaza Santa Ana 6, 28012 Madrid España",Bar,1.323 de 10.409,€€ - €€€,Plaza Santa Ana 6,Madrid España,Plaza Santa Ana 6,"Plaza Santa Ana 6, Madrid España","(40.38191525, -3.40964182827173)","Plaza Santa Ana 6, Madrid'","[-3.40964182827173, 40.38191525]","{'type': 'Point', 'coordinates': [-3.409641828...",-3.409642,40.381915


In [27]:

df = df.dropna(how = "any")

In [28]:
df.isnull().sum()

name              0
address           0
type              0
ranking           0
price             0
direccion         0
comunidad         0
street            0
street_2          0
coordinates       0
street_3          0
coordinates_v2    0
type_point        0
longitude         0
latitude          0
dtype: int64

In [29]:
df.price.unique()

array(['€€ - €€€', '€', '€€€€', 'Europea', 'Italiana'], dtype=object)

In [30]:
dic_for_price = {
    "€": "Barato",
    "€€ - €€€": "Medio",
    "€€€€": "Caro",
}

In [31]:
df['price_2'] = df.price.map(dic_for_price)

<ipython-input-31-8d0abb0c6806>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_2'] = df.price.map(dic_for_price)


In [32]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,longitude,latitude,price_2
indice,,,,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,549 de 10.409,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4...",-3.606505,40.028851,Medio
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,34 de 10.414,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ...",-3.590871,40.031684,Medio
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,650 de 10.414,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]","{'type': 'Point', 'coordinates': [-3.6142713, ...",-3.614271,40.039589,Barato
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,311 de 10.414,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]","{'type': 'Point', 'coordinates': [-3.6142713, ...",-3.614271,40.039589,Medio
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]","{'type': 'Point', 'coordinates': [-3.4252641, ...",-3.425264,40.138485,Medio


In [33]:
df.type.unique()

array(['Mediterránea', 'Bar', 'Restaurante de carne',
       'Confiterías/panaderías', 'Española', 'Europea', 'Argentina',
       'Internacional', 'Italiana', 'Asiática', 'Marisco', 'Caribeña',
       'Japonesa', 'Alemana', 'Americana', 'India', 'Postres',
       'Saludable', 'Café', 'Latina', 'Mexicana', 'Peruana', 'Tentempiés',
       'Belga', 'Filipina', 'Pizza', 'Comida rápida', 'Marroquí',
       'Cubana', 'Pub con cerveza artesanal', 'Fusión', 'Portuguesa',
       'Libanesa', 'Francesa', 'Brasileña', 'China', 'Salvadoreña'],
      dtype=object)

In [34]:
df.type.value_counts()

Mediterránea                 142
Italiana                      53
Bar                           35
Española                      34
Japonesa                      28
Internacional                 24
Restaurante de carne          22
Postres                       20
Americana                     19
Marisco                       18
Mexicana                      17
Café                          12
India                          9
Europea                        9
Tentempiés                     8
Confiterías/panaderías         7
Comida rápida                  6
Latina                         5
Asiática                       4
Peruana                        4
Caribeña                       3
Pub con cerveza artesanal      2
Francesa                       2
Fusión                         2
China                          1
Brasileña                      1
Libanesa                       1
Portuguesa                     1
Filipina                       1
Cubana                         1
Marroquí  

In [35]:
df["type"] = df["type"].str.replace("Pub con cerveza artesanal", "Bar")
df["type"] = df["type"].str.replace("Restaurante de carne", "Asador")
df["type"] = df["type"].str.replace("Comida rápida", "Americana")
df["type"] = df["type"].str.replace("Pizza", "Italiana")

df["type"] = df["type"].str.replace("Café", "Café y postres")
df["type"] = df["type"].str.replace("Confiterías/panaderías", "Café y postres")
df["type"] = df["type"].str.replace("Postres", "Café y postres")

df["type"] = df["type"].str.replace("Española", "Mediterránea")
df["type"] = df["type"].str.replace("Portuguesa", "Mediterránea")
df["type"] = df["type"].str.replace("Libanesa", "Mediterránea")

df["type"] = df["type"].str.replace("China", "Asiática")
df["type"] = df["type"].str.replace("Japonesa", "Asiática")

df["type"] = df["type"].str.replace("Tentempiés", "Otros")
df["type"] = df["type"].str.replace("Saludable", "Otros")

df["type"] = df["type"].str.replace("Francesa", "Europea")
df["type"] = df["type"].str.replace("Alemana", "Europea")
df["type"] = df["type"].str.replace("Belga", "Europea")

df["type"] = df["type"].str.replace("Peruana", "Latina")
df["type"] = df["type"].str.replace("Caribeña", "Latina")
df["type"] = df["type"].str.replace("Brasileña", "Latina")
df["type"] = df["type"].str.replace("Argentina", "Latina")
df["type"] = df["type"].str.replace("Salvadoreña", "Latina")
df["type"] = df["type"].str.replace("Cubana", "Latina")

<ipython-input-35-39beb05f1709>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["type"] = df["type"].str.replace("Pub con cerveza artesanal", "Bar")
<ipython-input-35-39beb05f1709>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["type"] = df["type"].str.replace("Restaurante de carne", "Asador")
<ipython-input-35-39beb05f1709>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [36]:
df.type.unique()

array(['Mediterránea', 'Bar', 'Asador', 'Café y postres', 'Europea',
       'Latina', 'Internacional', 'Italiana', 'Asiática', 'Marisco',
       'Americana', 'India', 'Otros', 'Mexicana', 'Filipina', 'Marroquí',
       'Fusión'], dtype=object)

In [ ]:
#df.to_csv('tripadvisor_latlon.csv')

In [39]:
!pwd

/Users/awalsh/IRONHACK/PROJECT2/Notebooks


In [38]:
df.to_csv('tripadvisor_latlon_v2.csv')

In [49]:
df.type.unique()

array(['Mediterránea', 'Bar', 'Asador', 'Café y postres', 'Europea',
       'Latina', 'Internacional', 'Italiana', 'Asiática', 'Marisco',
       'Americana', 'India', 'Otros', 'Mexicana', 'Filipina', 'Marroquí',
       'Fusión'], dtype=object)

## Exportar a Mongo

In [40]:
client = MongoClient("localhost:27017")

In [41]:
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [42]:
collection = db["Restaurants"]

In [ ]:
collection.create_index([("type_point_", "2dsphere")])

In [44]:
data = df.to_dict(orient='records')

In [45]:
df.type_point[0]

{'type': 'Point', 'coordinates': [-3.679035, 40.4243394]}

In [46]:
#collection.insert_many(data)

In [ ]:
ubicacion = input("¿Dónde estás?: ")
print(ubicacion)

In [ ]:
tipo = input("¿Qué tipo de comida te apetece?: ")
print(tipo)

In [ ]:
distance = int(input("¿Cuántos metros quieres desplazarte?: "))
print(distance)

In [ ]:
def get_coordenadas(address):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    geolocator = Nominatim(user_agent="Ana")
    location = geolocator.geocode(query=address, exactly_one=True)
    localizacion = location[1]
    return [localizacion[1], localizacion[0]]

In [ ]:
coordinate = get_coordenadas(ubicacion)

In [ ]:
coordinate

In [ ]:
inicial_lat = coordinate[1]
print(inicial_lat)
inicial_long =  coordinate[0]

In [ ]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [ ]:
db

## Restaurantes a 500 m

In [50]:
restaurants = db.get_collection("Restaurants")

In [51]:
all_restaurants = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}}}

NameError: name 'coordinate' is not defined

In [52]:
(list(restaurants.find(all_restaurants)))

NameError: name 'all_restaurants' is not defined

In [ ]:
df_restaurantes= pd.DataFrame(list(restaurants.find(all_restaurants)))

In [ ]:
df_restaurantes

In [ ]:
df_restaurantes.type.unique()

## Restaurantes Mediterraneos

In [ ]:
rest_mediterraneos = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Mediterránea"}

In [ ]:
rest_mediterraneos

In [ ]:
len(list(restaurants.find(query_restaurants)))

In [ ]:
df_medit = pd.DataFrame(list(restaurants.find(query_restaurants)))

In [ ]:
df_medit

## Restaurantes Española

In [ ]:
rest_internacional = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Internacional"}

In [ ]:
rest_internacional

In [ ]:
len(list(restaurants.find(rest_internacional)))

In [ ]:
df_intern = pd.DataFrame(list(restaurants.find(rest_internacional)))

In [ ]:
df_intern

## Maps

In [ ]:
df_restaurantes.head()

In [ ]:
def get_latitude (x):
    return x[0]

In [ ]:
def get_longitude (x):
    return x[1]

In [ ]:
df_restaurantes["latitud"] = df_restaurantes["coordinates"].apply(get_latitude)

In [ ]:
df_restaurantes["longitude"] = df_restaurantes["coordinates"].apply(get_longitude)

In [ ]:
df_restaurantes

In [ ]:
map_1 = folium.Map(location=[inicial_lat,inicial_long], zoom_start=15)

dic = {"location": [inicial_lat, inicial_long], "tooltip": "ubicacion"}

icono = Icon(color = "red",
                     prefix="fa",
                     icon="map-marker",
                     icon_color="black")

mark = Marker(**dic, icon=icono)
mark.add_to(map_1)

In [ ]:
map_1

In [ ]:
df_restaurantes.type.unique()

In [ ]:
for i,row in df_restaurantes.iterrows():

    #dic = {"location": [row["latitud"], row["longitud"]], "tooltip": row["nombre"]}
    dic = {"location": [row["latitud"], row["longitude"]], "tooltip": row["name"]}

    if row["type"] == "Mediterránea":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

    elif row["type"] == 'Italiana':
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

        
    elif row["type"] == "Internacional":        
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Confiterías/panaderías":        
        icono = Icon(color = "gray",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Japonesa":        
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Bar":        
        icono = Icon(color = "lightgreen",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "India":        
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
                 
           
    elif row["type"] == "Mexicana":        
        icono = Icon(color = "pink",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Pub con cerveza artesanal":        
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    
    mark = Marker(**dic, icon=icono)
    mark.add_to(map_1)   

In [ ]:
map_1

1- Input preguntando donde estas
2- Input preguntando que radio quieren
3- Funcion que pasa de direccion a coordenadas (coordenadas al reves)
4- Funcion que me haga una query near a moongo y me devuelva todos los restaurantes en el radio  




streamlit : parte del csv, estoy en tal sitio y quiero tal tipo de comida en esta distancia.

- tendre una funcion que haga la query y saque un pandas. support
- tendre una función que haga un mapa con un pd. return map. spport


- df = función(input del usuario)
- map (df)




mañana hay que tener la presentación.

1- mapas (aqui va el input del usuario)
para la distancia hago un sidebar. esta en iris_prediction/src/ support.py /

2 - graficos


3- diccionario map sobre el pd.





